In [1]:
# Standard python code
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import c
from scipy.special import fresnel
import sys
sys.path.insert(0,"../")
# huntstone imports

from plotting import makefig
import prop_plasma as pp
from read_data import read_data
import spectral_intensity as si
from v_x_interp import velocity_interp, position_interp

# Plotting style
plt.style.use('huntstone')
# Set parameters for reading WARGSim dumps
path   = "/media/keenan/Data_Storage/WARGSim/500dumps/"
N      = int(1e6)
nDumps = 699

beam_params = {
    'N'       : int(1e6),
    'beamE'   : 10e9, 
    'eps_n0'  : 3.0e-6,
    'beta0'   : [0.25, 0.25], 
    'alpha0'  : [0.00, 0.00],
    'rms_z'  : 0.00,
    'rms_gb0' : 0.01,
    'path'    : '/media/keenan/Data_Storage/WARGSim/',
    'B_mag'   : 5.00
}
plasma_params = {
    'n0'    : 0.34,
    'L_ft'  : 0.4,
    'hw_up' : 0.0,
    'shape' : 'Gauss'
}
my_ebeam = pp.init_beam(beam_params, plasma_params['n0'])
pwfa0    = pp.init_plasma(beam_params, plasma_params)
dumpPeriod = int(len(pwfa0.dz) / nDumps)

beta factor = 43.83
Bmag = 21.92
init eps_nx =  3.0026053450567585e-06
init eps_ny =  2.9995423655595026e-06


In [2]:
# Get positions and times
x = np.load("x_noGain.npy")
y = np.load("y_noGain.npy")
gb = np.load("gb_noGain.npy")
z = np.linspace(0, pwfa0.L, nDumps)
t = z / c
# Calculate proper time
dtau = np.diff(t)[0]/gb
tau  = np.zeros(nDumps)
for i in range(1, nDumps):
    tau[i] = tau[i-1] + dtau[i-1]
vx  = np.diff(x) / np.diff(tau)
vx  = np.append(vx, vx[-1])
vy  = np.diff(y) / np.diff(tau)
vy  = np.append(vy, vy[-1])
vz  = np.diff(z) / np.diff(tau)
vz  = np.append(vz, vz[-1])
vct = c * gb 
x4  = (c*t, x, y, z)
v4  = (vct, vx, vy, vz)
v   = (vx, vy, vz)
tau_int = np.linspace(tau[0], tau[-1], 1000)

In [4]:
s = [1/2, 1/2, 1/np.sqrt(2)]
w = 1e12
si.get_intensity(s, w, x4, v4, tau, tau_int, dtau)

2.7189800194635703e-22